In [4]:
import pandas as pd

def check_duplicates(df, file_name):
    """Check for duplicate offer_ids in a DataFrame and print results"""
    duplicates = df[df.duplicated(subset=['offer_id'], keep=False)]
    if len(duplicates) > 0:
        dup_count = len(duplicates['offer_id'].unique())
        total_dups = len(duplicates)
        print(f"⚠️ WARNING: {file_name} has {total_dups} duplicate rows ({dup_count} unique offer_ids)")
        print(f"Example duplicates: {duplicates['offer_id'].unique()[:5].tolist()}")
        return True
    else:
        print(f"✓ {file_name} has no duplicate offer_ids")
        return False

def merge_csv_files(handle_duplicates=True):
    """
    Merge specified CSV files using an outer join on offer_id with duplicate handling.
    
    Args:
        handle_duplicates: If True, removes duplicate offer_ids from source files before merging
        
    Returns:
        pd.DataFrame: Merged DataFrame containing data from all input CSV files
    """
    # Define file list and handling functions
    file_list = [
        'features.csv',
        'estimation.csv',
        'stats.csv',
        'apartment_details.csv',
        'building_details.csv', 
        'rental_terms.csv',
        'cian_apartments.csv'
    ]
    
    # Check all files for duplicates first
    print("Checking files for duplicate offer_ids:")
    for file_name in file_list:
        df = pd.read_csv(file_name)
        has_dups = check_duplicates(df, file_name)
    
    # Start with base file
    df_features = pd.read_csv('features.csv')
    if handle_duplicates and check_duplicates(df_features, 'features.csv'):
        df_features = df_features.drop_duplicates(subset=['offer_id'], keep='first')
    merged_df = df_features
    print(f"Loaded features.csv with {len(merged_df)} rows and {len(merged_df.columns)} columns")
    
    # Merge estimation.csv
    df_estimation = pd.read_csv('estimation.csv')
    if handle_duplicates and check_duplicates(df_estimation, 'estimation.csv'):
        df_estimation = df_estimation.drop_duplicates(subset=['offer_id'], keep='first')
    merged_df = pd.merge(merged_df, df_estimation, on='offer_id', how='outer',
                        suffixes=('', '_estimation'))
    print(f"After merging estimation.csv: {len(merged_df)} rows and {len(merged_df.columns)} columns")
    
    # Merge stats.csv
    df_stats = pd.read_csv('stats.csv')
    if handle_duplicates and check_duplicates(df_stats, 'stats.csv'):
        df_stats = df_stats.drop_duplicates(subset=['offer_id'], keep='first')
    merged_df = pd.merge(merged_df, df_stats, on='offer_id', how='outer',
                        suffixes=('', '_stats'))
    print(f"After merging stats.csv: {len(merged_df)} rows and {len(merged_df.columns)} columns")
    
    # Merge apartment_details.csv
    df_apartment = pd.read_csv('apartment_details.csv')
    if handle_duplicates and check_duplicates(df_apartment, 'apartment_details.csv'):
        df_apartment = df_apartment.drop_duplicates(subset=['offer_id'], keep='first')
    merged_df = pd.merge(merged_df, df_apartment, on='offer_id', how='outer',
                        suffixes=('', '_apartment'))
    print(f"After merging apartment_details.csv: {len(merged_df)} rows and {len(merged_df.columns)} columns")
    
    # Merge building_details.csv
    df_building = pd.read_csv('building_details.csv')
    if handle_duplicates and check_duplicates(df_building, 'building_details.csv'):
        df_building = df_building.drop_duplicates(subset=['offer_id'], keep='first')
    merged_df = pd.merge(merged_df, df_building, on='offer_id', how='outer',
                        suffixes=('', '_building'))
    print(f"After merging building_details.csv: {len(merged_df)} rows and {len(merged_df.columns)} columns")
    
    # Merge rental_terms.csv - with specific suffix to handle duplicate columns
    df_terms = pd.read_csv('rental_terms.csv')
    if handle_duplicates and check_duplicates(df_terms, 'rental_terms.csv'):
        df_terms = df_terms.drop_duplicates(subset=['offer_id'], keep='first')
    merged_df = pd.merge(merged_df, df_terms, on='offer_id', how='outer',
                        suffixes=('', '_terms'))
    print(f"After merging rental_terms.csv: {len(merged_df)} rows and {len(merged_df.columns)} columns")
    
    # Merge cian_apartments.csv last as it has several duplicate columns
    df_cian = pd.read_csv('cian_apartments.csv')
    if handle_duplicates and check_duplicates(df_cian, 'cian_apartments.csv'):
        df_cian = df_cian.drop_duplicates(subset=['offer_id'], keep='first')
    merged_df = pd.merge(merged_df, df_cian, on='offer_id', how='outer',
                        suffixes=('', '_cian'))
    print(f"After merging cian_apartments.csv: {len(merged_df)} rows and {len(merged_df.columns)} columns")
    
    # Clean up column names - remove any empty suffixes that might have been added
    merged_df.columns = [col.replace('_', '') if col.endswith('_') else col 
                         for col in merged_df.columns]
    
    return merged_df

def main():
    # Let user know what's happening
    print("\nRunning merge process with duplicate handling...\n")
    
    # Merge the CSV files with duplicate handling
    merged_dataframe = merge_csv_files(handle_duplicates=True)
    
    # Check for any remaining duplicates in the final dataset
    print("\nChecking for duplicates in final merged dataset:")
    duplicates = merged_dataframe[merged_dataframe.duplicated(subset=['offer_id'], keep=False)]
    if len(duplicates) > 0:
        dup_count = len(duplicates['offer_id'].unique())
        total_dups = len(duplicates)
        print(f"⚠️ WARNING: Final dataset still has {total_dups} duplicate rows ({dup_count} unique offer_ids)")
        
        # Print a sample of the duplicate records
        print("\nSample of duplicate offer_ids in final dataset:")
        example_id = duplicates['offer_id'].iloc[0]
        print(f"Records for offer_id {example_id}:")
        print(merged_dataframe[merged_dataframe['offer_id'] == example_id].head(2))
    else:
        print("✓ Final dataset has no duplicate offer_ids")
    
    # Print a list of columns with suffixes to help identify duplicates
    suffix_columns = [col for col in merged_dataframe.columns if '_terms' in col or '_cian' in col 
                      or '_estimation' in col or '_stats' in col or '_apartment' in col 
                      or '_building' in col]
    if suffix_columns:
        print("\nColumns with suffixes (indicating duplicates):")
        for col in suffix_columns:
            print(f"- {col}")
    
    # Save the merged DataFrame
    merged_dataframe.to_csv('merged_apartments_data.csv', index=False, encoding='utf-8')
    print("\nMerged data saved to 'merged_apartments_data.csv'")
    
    # Display information about the merged dataset
    print("\nMerged Dataset Information:")
    print(f"Total number of rows: {len(merged_dataframe)}")
    print(f"Total number of columns: {len(merged_dataframe.columns)}")
    
    # Print all column names
    print("\nColumns in the merged dataset:")
    print(merged_dataframe.columns.tolist())
    
    # Optional: Preview the data
    print("\nPreview of merged data (first 5 rows):")
    print(merged_dataframe.head(5))

if __name__ == '__main__':
    main()

Loaded features.csv with 1031 rows and 11 columns
After merging estimation.csv: 1033 rows and 13 columns
After merging stats.csv: 1033 rows and 21 columns
After merging apartment_details.csv: 3513 rows and 32 columns
After merging building_details.csv: 16945 rows and 43 columns
After merging rental_terms.csv: 102681 rows and 50 columns
After merging cian_apartments.csv: 102777 rows and 74 columns

Columns with suffixes (indicating duplicates):
- rental_period_cian

Merged Dataset Information:
Total number of rows: 102777
Total number of columns: 74

Columns in the merged dataset:
['offer_id', 'has_refrigerator', 'has_dishwasher', 'has_washing_machine', 'has_air_conditioner', 'has_tv', 'has_internet', 'has_kitchen_furniture', 'has_room_furniture', 'has_bathtub', 'has_shower_cabin', 'estimated_price', 'estimated_price_clean', 'creation_date', 'creation_date_iso', 'updated_date', 'updated_date_iso', 'total_views', 'recent_views', 'unique_views', 'is_unpublished', 'layout', 'apartment_type